In [1]:
import spacy
from allennlp.predictors.predictor import Predictor
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
predictor=Predictor.from_path("../model/structured-prediction-srl-bert.2020.12.15.tar.gz")

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [15]:
class Entity():
    def __init__(self,text,type,start,end) -> None:
        self.text = text
        self.type = type
        self.start_offset=start
        self.end_offset = end
def Get_arguments(tokens,srl_dict,event_start_offset,event_end_offset,entities):
    idx_mapping = {}
    i,j = 0,0
    srl_tokens = srl_dict['words']
    while i < len(tokens):
        token = tokens[i]
        while True:
            if token.startswith(srl_tokens[j]):
                start_idx = j
                token_srl = srl_tokens[j]
                j += 1
                break
            else:
                j += 1
        while token != token_srl and j < len(srl_tokens):
            if token.startswith((token_srl+srl_tokens[j])):
                token_srl += srl_tokens[j]
            j += 1
        idx_mapping[i] = (start_idx, j)#空格分的token与预测器分的token的对应关系
        i += 1
    arg0,arg1,loc,time=(-1,-1),(-1,-1),(-1,-1),(-1,-1),

    #这里是ecb的逻辑，触发词作为事件mention,但是wec应该怎么标，只是一个事件mention，参数信息大多并不是描述这个时间的
    for verb in srl_dict['verbs']:
        flag = False
        for i in range(idx_mapping[event_start_offset][0], idx_mapping[event_end_offset][1]):

            if '-V' in verb['tags'][i]:  #
                flag = True
                break
        if not flag:
            continue

        for entity in entities:
            for i in range(idx_mapping[entity.start_offset][0], idx_mapping[entity.end_offset][1]):
                if verb['tags'][i] != 'O':
                    type = verb['tags'][i].split('-')[-1]
                    if type == 'ARG0' and 'GPE' not in entity.type and 'DATE' not in entity.type:
                        arg0 = (entity.start_offset, entity.end_offset)
                        break
                    if type == 'ARG1' and 'GPE' not in entity.type and 'DATE' not in entity.type:
                        arg1 = (entity.start_offset, entity.end_offset)
                        break
                    if type == 'GPE' and 'GPE' in entity.type:
                        loc = (entity.start_offset, entity.end_offset)
                        break
                    if type == 'TMP' and 'DATE' in entity.type:
                        time = (entity.start_offset, entity.end_offset)
                        break
    return arg0,arg1,loc,time
def Get_entities(doc,tokens):
    i=0
    entities=[]
    for entity in doc.ents:#entity是按顺序给出的
        start,end=-1,-1
        flag = True
        while(i<len(tokens) and flag):
            tokens[i]
            if entity.text.startswith(tokens[i]) or tokens[i].startswith(entity.text.split(' ')[-1]):
                start = i
                # palo alto ->palo alto-based
                while True:
                    if entity.text.endswith(tokens[i]) or tokens[i].startswith(entity.text.split(' ')[-1]):
                        end = i
                        flag=False#标记当前entity已找到
                        break
                    i+=1
            i+=1
        entities.append(Entity(entity.text,entity.label_,start,end)) #不能用这个start，end 并不是按空格划分的
    return entities

In [ ]:
# It appears to be a very daunting deal , '' said American Technology Research analyst Shaw Wu .	11	11	16	17	-1	-1	-1	-1	-1	-1	
# Palo Alto-based HP and Plano , Texas-based EDS confirmed the talks yesterday shortly after The Wall Street Journal reported a deal could be reached as early as today .	23	23	-1	-1	-1	-1	-1	-1	27	27	0

In [27]:
text1 = " '' It appears to be a very daunting deal , '' said American Technology Research analyst Shaw Wu ."
text2 = "Palo Alto-based HP and Plano ,yestarday Seven in the morning in India,Texas-based EDS confirmed the talks yesterday shortly after The Wall Street Journal reported a deal could be reached as early as today ."
tokens1 = text1.strip().split(" ")
tokens2 = text2.strip().split(" ")
doc1 = nlp(text1)
doc2 = nlp(text2)
entities1=Get_entities(doc1,tokens1)
entities2=Get_entities(doc2,tokens2)
# Palo Alto-based 标注的是palo alto  但我应该在这个将实体定义为 palo alto-based，这样在tokenizer时也能拼起来

src1_result = predictor.predict(sentence=text1)
src2_result = predictor.predict(sentence=text2)

In [9]:
text1_arg0,text1_arg1,text1_loc,text1_time =Get_arguments(tokens1,src1_result,11,11,entities1)
print(text1_arg0,text1_arg1,text1_loc,text1_time)
text2_arg0,text2_arg1,text2_loc,text2_time =Get_arguments(tokens2,src2_result,23,23,entities2)
print(text2_arg0,text2_arg1,text2_loc,text2_time)

(16, 17) (-1, -1) (-1, -1) (-1, -1)
(-1, -1) (-1, -1) (-1, -1) (19, 27)


In [ ]:

print(src1_result,src2_result)


In [11]:
for entity in entities2:
    print(entity.text,entity.type,entity.start_offset,entity.end_offset)

Palo Alto GPE 0 1
HP and Plano ORG 2 4
Texas GPE 6 6
EDS ORG 7 7
yesterday DATE 11 11
The Wall Street Journal ORG 14 17
as early as today DATE 19 27


In [28]:
for entity in entities2:
    print(entity.text,entity.type,entity.start_offset,entity.end_offset)

Palo Alto GPE 0 1
HP and Plano ORG 2 4
Seven in the morning TIME 6 9
India GPE 11 11
Texas GPE -1 -1
EDS ORG -1 -1
yesterday DATE -1 -1
The Wall Street Journal ORG -1 -1
as early as today DATE -1 -1


In [25]:
text1 = "'' It appears to be a very daunting deal , '' said American Technology Research analyst Shaw Wu ."
tokens1 = text1.split(" ")
print(tokens1)
doc1 = nlp(text1)
for entity in doc1.ents:
    print(entity.text,entity.label_,entity.start,entity.end)

["''", 'It', 'appears', 'to', 'be', 'a', 'very', 'daunting', 'deal', ',', "''", 'said', 'American', 'Technology', 'Research', 'analyst', 'Shaw', 'Wu', '.']
American Technology Research ORG 12 15
Shaw Wu PERSON 16 18


d:\download\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [51]:
import torch
import numpy as np
from scipy.special import softmax
a = torch.rand((4,2),dtype=torch.float)
b = torch.rand((4,2),dtype=torch.float)
preds=[]
preds.append(a)
preds[0] = np.append(preds[0], b, axis=0)

preds = preds[0]

pred_probs = softmax(preds,axis=1)
score_for_print = list(pred_probs[:,0])

pred_label_ids = list(np.argmax(pred_probs, axis=1))



In [60]:
pred_label_ids
labels=[1,1,0,0,0,0,1,1]
predictions=pred_label_ids
pred_label_ids
probs=torch.rand((8,1))
predictions = torch.where( probs> 0.5, 1.0, 0.0)
print(probs,predictions)

tensor([[0.2449],
        [0.3374],
        [0.5047],
        [0.2891],
        [0.1189],
        [0.9249],
        [0.7378],
        [0.3168]]) tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]])


False

In [5]:
import json
Metric_dict={'a':1,'b':[1,3,'a']}
with open("./result/ecb+_metrics.txt",'a',encoding='utf-8')as f:
        json.dump(Metric_dict,f,ensure_ascii=False)
        f.write(',\n')
        f.close()


In [33]:
l =torch.tensor((1,2,3))
l.tolist()
l


tensor([1, 2, 3])